# Jornada Paulista de Radiologia 2018

# Hands-on - Deep Learning para detectar hemorragia intracraniana



**Desenvolvido por:**

Felipe Campos Kitamura (kitamura.felipe@gmail.com)

Igor Santos (igorafael87@gmail.com)

Thiago Julio

Luciano Prevedello


**Instrutores:**

Marcelo Froeder

Daisy Kase

José Eduardo Venson


**Agradecimento:**

Marcelo Felix



**OBJETIVO:**

Demonstrar os passos básicos do treinamento de uma rede neural convolucional para detectar hemorragia intracraniana.


**Passo a passo:**

Todo o processo será realizado com a linguagem de programação Python, versão 3.

O dataset utilizado consiste em cerca de 100 imagens de tomografias de crânio normais e 100 imagens com sangramento intracraniano.

Para tarefas específicas importaremos bibliotecas específicas.

# Passo 1: criando o dataset

In [0]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
#Primeiro vamos baixar o nosso dataset. Você pode baixar esse arquivo no seu computador para ver as imagens que utilizaremos nesse hands-on.
!wget https://github.com/igorafaelms/deeplearningddi/blob/master/Cranio.zip?raw=true

print ('\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

In [0]:
#Agora vamos extrair os arquivos de imagens de dentro desse arquivo .zip
!unzip -o Cranio.zip?raw=true -d /

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

In [0]:
#Para abrir as imagens .jpg, utilizaremos o glob, que lista os arquivos que existem dentro de uma pasta

from glob import glob

print ('\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Para executar cada célula, pressione SHIFT + ENTER

In [0]:
#Vamos definir o caminho das pastas que contém as imagens
hematoma_dir = '/Cranio/Hematoma/*.png'
normal_dir = '/Cranio/Normal/*.png'

#Agora vamos listar os arquvos dentro de cada uma das pastas, usando o glob()
hematoma_lista = glob(hematoma_dir)
normal_lista = glob(normal_dir)

#Para termos ideia do número de arquivos em cada pasta, vamos dar um print(len(lista)))
print('Número de casos com hematoma: ', len(hematoma_lista))
print('Número de casos sem hematoma: ', len(normal_lista))

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Execute esse código com SHIFT + ENTER

#A saída esperada é

#Número de casos com hematoma:  100
#Número de casos sem hematoma:  100

In [0]:
#Para ter ideia de como é a lista de arquivos gerada pelo glob, vamos olhar a variável hematoma_lista
hematoma_lista
print(*hematoma_lista, sep = "\n")

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Execute esse código com SHIFT + ENTER

**Como o computador enxerga uma imagem?**

Considere cada imagem como uma matriz em que o valor de cada pixel corresponde a um 
número que determina o tom de cinza da imagem.


In [0]:
#Podemos também ver cada uma das imagens do nosso dataset. Para isso, precisaremos abrir um aquivo .png e plotá-lo num gráfico.

#Para abrir o arquivo de imagem, utilizaremos o openCV, uma biblioteca aberta de visão computacional

import cv2

#Utilizaremos uma biblioteca de plotagem de gráficos chamada a matplotlib

from matplotlib import pyplot as plt

#Mãos a obra, vamos abrir um arquivo como exemplo:

ID_arquivo = 4

imagem = cv2.imread(hematoma_lista[ID_arquivo])

#Esse comando é só para ler o arquivo. Agora vamos criar uma figura com a imagem que abrimos (plotar a imagem):

plt.imshow(imagem)
plt.show()

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')


#Execute esse código com SHIFT + ENTER

#Tente modificar ID_arquivo com outros números para vizualizar outras imagens. Repare no tamanho de cada imagem

In [0]:
#Repare que cada imagem tem um tamanho diferente. 

#Como as redes neurais tem entrada de tamanho fixo, precisaremos redimensionar todas elas para um tamanho único.

#Além disso, precisamos informar explicitamente a qual das categorias as imagens pertecem em uma lista chamada labels (hematoma = 1, normal = 0)

#Faremos isso ao mesmo tempo que salvaremos as imagens numa matriz. Para usar matrizes, importaremos a NumPy.

# Por que usamos matrizes? A entrada de informações nas redes neurais se dá nesse formato,
# pois permite processamento computacional paralelo e maior velocidade de processamento.

import numpy as np

dataset = [] # cria uma lista vazia para incluir as imagens do dataset
labels = [] # cria uma lista vazia para incluir a categoria a qual cada imagem pertence (0 ou 1)

for arquivo in hematoma_lista: # para cada arquivo de imagem na lista hematoma:
    img = cv2.imread(arquivo, cv2.IMREAD_GRAYSCALE) #abre o arquivo como escala de cinzas e coloca a imagem na variável img
    img = cv2.resize(img, (256,256)) #redimensiona a imagem para 256 x 256 e salva na mesma variável img
    dataset.append(img) #adiciona essa imagem na lista do dataset e 
    labels.append(1) #informa que ela é um caso de hematoma (1)

#Agora faremos o mesmo para as imagens sem sangramento

for arquivo in normal_lista: # para cada arquivo de imagem na lista hematoma:
    img = cv2.imread(arquivo, cv2.IMREAD_GRAYSCALE) #abre o arquivo como escala de cinzas e coloca a imagem na variável img
    img = cv2.resize(img, (256,256)) #redimensiona a imagem para 256 x 256 e salva na mesma variável img
    dataset.append(img) #adiciona essa imagem na lista do dataset e 
    labels.append(0) #informa que ela é um caso normal (0)
    
dataset = np.asarray(dataset, dtype=np.float32) #transforma a lista de variáveis numa matriz
labels = np.asarray(labels)

for i in range(len(dataset)):
  dataset[i] = (dataset[i] - np.average(dataset[i], axis= (0, 1))) / np.std(dataset[i], axis= (0, 1))

#Vamos ver qual o tamanho dessa matriz 'dataset'

#Esperamos que a primeira dimensão dela seja de 200 (100 casos de hematoma e 100 normais)

#A segunda e a terceira dimensões devem ser 256.

#Execute esse código com SHIFT + ENTER
print(dataset.shape)

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

# a saída esperada é (200, 256, 256)

# Colocar exemplo de uma matriz

# Passo 2: criando a estrutura da rede neural convolucional

Uma convolução é uma operação matemátca que consiste em multiplicar uma matriz (a nossa imagem) por um 
filtro (matrix de pesos)


In [0]:
#Para criar a rede neural convolucional, utilizaremos o Keras, que é uma biblioteca para Deep Learning em Python

#Inicialmente vamos importar as funções do Keras que iremos utilizar:

from keras.models import Sequential
from keras import optimizers
from keras.layers.core import Dense, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D, GlobalAveragePooling2D
from keras.models import Model
from keras.layers import Input, Concatenate, add
from keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Dense, LeakyReLU
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

#Agora criaremos a estrutura da rede neural convolucional

#Essa é a definição da camada de entrada da rede
#Lembra que a nossa matriz com todas as imagens de cada categoria tem o formato (200, 256, 256)?
#Nesse caso a entrada (input) da rede é cada imagem individualmente
#Ou seja, uma imagem de tamanho 256 x 256 pixels e 1 canal de cor (escala de cinzas)

imgs = Input(shape=(256,256,1))

#Abaixo temos a primeira camada Convolucional


x = Conv2D(8, 3, padding='same', activation='relu')(imgs)

#Em seguida, adicionamos uma camada MaxPool, que irá reduzir em 75% o tamanho da saída da camada convolucional.
#Fazemos isso para evitar que o número de parâmetros da rede aumente demais.
#Não se preocupe em entender o detalhe de cada uma dessas operações. Tente captar a ideia geral.
x = MaxPool2D()(x)

#Adicionaremos mais camadas convolucionais, seguidas de MaxPool

x = Conv2D(8, 3, padding='same', activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(8, 3, padding='same', activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(8, 3, padding='same', activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(8, 3, padding='same', activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(8, 3, padding='same', activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(10, 3, padding='same', activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(10, 3, padding='same', activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(12, 3, padding='same', activation='relu')(x)
x = Conv2D(12, 3, padding='same', activation='relu')(x)
x = Conv2D(12, 3, padding='same', activation='relu')(x)
x = GlobalAveragePooling2D()(x)

#Finalmente adicionaremos duas camadas densas, ou 'Fully Connected Layers'.
#Essas camadas são redes neurais convencionais, sem convolução.
#Não se preocupe com o porquê de usarmos essa camada agora.
x = Dense(64, activation='relu')(x)

#Dropout é uma maneira de reduzir overfitting.
x = Dropout(0.6)(x)

#Definiremos agora a entrada e a saída da rede
#A função Dense tem o argumento "1" pois a saída da rede é a classificação hematoma x não-hematoma
#Ou seja, a saída da rede é apenas um número (0 ou 1)

x = Dense(32, activation='relu')(x)

outputs = Dense(1, activation='sigmoid')(x)

inputs = imgs

#Por fim, definimos a rede

Rede_JPR = Model(inputs=inputs, outputs=outputs)

#Agora, definiremos o método de otimização da rede: ADAM, com a taxa de aprendizado e de decaimento
#Cada um desses parâmetros é ajustável.

custom_adam = optimizers.Adam(lr=0.0005, decay=0.0002)

#Compila o modelo
Rede_JPR.compile(loss='binary_crossentropy', optimizer=custom_adam, metrics=['acc'])

print ('\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Execute esse código com SHIFT + ENTER

# Passo 3: Dividindo o dataset nos grupos Treinamento/Validação/Teste

In [0]:
#Vamos separar nosso dataset em grupos de treinamento, validação e test. Para isso, usaremos a biblioteca sklearn.

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#Divide treino, validação e teste na proporção 80%/10%/10%

dataset_train, dataset_test, labels_train, labels_test = train_test_split(dataset[:,...,np.newaxis], labels[:,...,np.newaxis], test_size=0.1, random_state=88)
dataset_train, dataset_val, labels_train, labels_val = train_test_split(dataset_train, labels_train, test_size=0.11, random_state=88)

#Mostra o tamanho das variáveis dos grupos

print('(Número de imagens, Imagem_X, Imagem_Y, canais de cor) (Número de labels, 1)')
print(dataset_train.shape, labels_train.shape)
print(dataset_val.shape, labels_val.shape)
print(dataset_test.shape, labels_test.shape)

#Você deve ver a seguinte saída:

#(160, 256, 256, 1) (160,1)
#(20, 256, 256, 1) (20,1)
#(20, 256, 256, 1) (20,1)

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Execute esse código com SHIFT + ENTER

# Passo 4: como treinar a sua rede neural

Durante o treinamento da rede, os pesos das sinapses de todas as camadas são atualizados.

Após a conclusão do treinamento, cada filtro representará uma característica de imagem a ser procurada nas imagens do nosso dataset.

In [0]:
#Hora de treinar a nossa rede neural.

#Primeiro vamos definir para salvar o melhor modelo que for encontrado durante o treino

checkpointer = ModelCheckpoint(filepath='Melhor_modelo.hdf5', monitor='val_loss',
                               verbose=1, save_best_only=True)

#Muito bem, chegou a hora mais esperada, vamos treinar a nossa rede com o dataset que criamos

print('Treinando a Rede_JPR:')


datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#datagen.fit(dataset_train)
  


#Definimos o treinamento com o dataset de treino, realizando validação no dataset de validação.
#O treinamento não usa o dataset de teste

Valida = (dataset_val, labels_val)

hist = Rede_JPR.fit_generator(datagen.flow(dataset_train, labels_train, batch_size=20), steps_per_epoch=1*len(dataset_train) / 20, epochs=200, 
                    validation_data= (dataset_val, labels_val), 
                    callbacks=[checkpointer])

#Por fim, plotamos os resultados de evolução da medida de erro (loss) ao longo das épocas

plt.plot(hist.history['loss'], 'b-', label='train loss')
plt.plot(hist.history['val_loss'], 'r-', label='val loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()


plt.plot(hist.history['acc'], 'b-', label='train accuracy')
plt.plot(hist.history['val_acc'], 'r-', label='val accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.show()

del(Rede_JPR)
print('Treino finalizado.')

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Execute esse código com SHIFT + ENTER

# Passo 5: avaliando sua rede no conjunto de Teste

Neste passo, iremos apresentar todo o dataset de treino para o modelo que criamos, de forma a calcular a acurácia da nossa rede neural no grupo de treino

In [0]:
#Vamos importar a biblioteca do keras que abre modelos salvos previamente
from keras.models import load_model

#Agora abrimos o melhor modelo que geramos anteriormente

melhor_modelo = load_model('Melhor_modelo.hdf5')

print ('\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Execute esse código com SHIFT + ENTER

In [0]:
#Usamos a função evaluate para avaliar a acurácia do nosso modelo no grupo de teste
print('Acurácia no grupo de teste: ', melhor_modelo.evaluate(dataset_test, labels_test, verbose=0)[1])

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Execute esse código com SHIFT + ENTER

**Podemos ver a predição caso a caso**



In [0]:
#Agora vemos fazer a inferência em imagens individuais do nosso dataset de teste.

#Defina uma imagem no grupo de Teste, de 0 a 19:

ID_imagem = 8

#Agora vamos criar uma figura com a imagem escolhemos

plt.imshow(np.squeeze(dataset_test[ID_imagem]), cmap='gray')
plt.show()

#Vamos mostrar a qual classe ela pertence

print('Classe:', 'normal' if labels_test[ID_imagem]==0 else 'hematoma')

predicao = np.round(melhor_modelo.predict(dataset_test[ID_imagem][np.newaxis,:,...], verbose=0))==0

print('Predição:', 'normal' if predicao else 'hematoma')

#Calcula o tempo médio de predição
print('Tempo médio de predição:')
%timeit melhor_modelo.predict(dataset_test[ID_imagem][np.newaxis,:,...], verbose=0)

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

#Execute esse código com SHIFT + ENTER

In [0]:
!pip3 install keras-vis

print ('\n' + '\033[1m' + 'Etapa Concluída. Vá para o próximo comando!')

In [0]:
#@title Default title text
from vis.visualization import saliency
from vis.visualization import activation_maximization
from vis.visualization.__init__ import get_num_filters

ID_imagem = 14

Layer = -17

#mapa = saliency.visualize_saliency(teste, Layer, filter_indices=range(get_num_filters(teste.layers[Layer])), seed_input=ds_test[ID_imagem], backprop_modifier=None, \
#    grad_modifier="absolute")

#mapa = activation_maximization.visualize_activation(teste, Layer, filter_indices=range(get_num_filters(teste.layers[Layer])), seed_input=ds_test[ID_imagem], \
#    backprop_modifier=None, grad_modifier=None, act_max_weight=1, lp_norm_weight=10, \
#    tv_weight=10)

mapa = saliency.visualize_cam(melhor_modelo, Layer, filter_indices=range(get_num_filters(melhor_modelo.layers[Layer])), seed_input=dataset_test[ID_imagem], penultimate_layer_idx=Layer-1, \
    backprop_modifier=None, grad_modifier=None)

plt.imshow(np.squeeze(dataset_test[ID_imagem]), cmap='gray')
plt.show()
plt.imshow(np.squeeze(mapa), cmap='jet')
plt.show()


ID_imagem = 1

Layer = -17

#mapa = saliency.visualize_saliency(teste, Layer, filter_indices=range(get_num_filters(teste.layers[Layer])), seed_input=ds_test[ID_imagem], backprop_modifier=None, \
#    grad_modifier="absolute")

#mapa = activation_maximization.visualize_activation(teste, Layer, filter_indices=range(get_num_filters(teste.layers[Layer])), seed_input=ds_test[ID_imagem], \
#    backprop_modifier=None, grad_modifier=None, act_max_weight=1, lp_norm_weight=10, \
#    tv_weight=10)

mapa = saliency.visualize_cam(melhor_modelo, Layer, filter_indices=range(get_num_filters(melhor_modelo.layers[Layer])), seed_input=dataset_test[ID_imagem], penultimate_layer_idx=Layer-1, \
    backprop_modifier=None, grad_modifier=None)

plt.imshow(np.squeeze(dataset_test[ID_imagem]), cmap='gray')
plt.show()
plt.imshow(np.squeeze(mapa), cmap='jet')
plt.show()

print ('\n' + '\033[1m' + 'Etapa Concluída. Parabéns, você finalizou o Workshop!')

from IPython.display import HTML
HTML('<img src="https://media.giphy.com/media/cub3pntkz8muQ/giphy.gif">')